In [1]:
pip install langchain langsmith langchain-openai openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 
import getpass
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACED_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Give me your langchain API Key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Give me your openai API Key: ")

model = ChatOpenAI(model="gpt-3.5-turbo")


Give me your langchain API Key: ········
Give me your openai API Key: ········


In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! My name is Kalki")])

AIMessage(content='Nice to meet you, Kalki! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 15, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b11d5e7a-f2eb-4ca7-92ed-5a9b052f42b9-0', usage_metadata={'input_tokens': 15, 'output_tokens': 16, 'total_tokens': 31})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-70b627b4-a379-45f8-8191-992594aa732a-0', usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49})

In [5]:
pip install langchain-community

  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.9 MB/s eta 0:00:002.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import  RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
config = {"configurable": {"session_id": "dub1"}}

In [13]:
response = with_message_history.invoke([HumanMessage(content="Hi, my name is Madhu and I prefer cats over dogs")], config=config,)
response.content

'Nice to meet you, Madhu! Cats are wonderful companions. What do you love most about them?'

In [14]:
response = with_message_history.invoke([HumanMessage(content="What do i prefer over dogs exactly and how am I?")], config=config)
response.content

"It seems like you prefer cats over dogs. As for how you are, I'm not sure as I don't have personal information about you. Can you tell me more about yourself so I can get to know you better?"

In [15]:
response = with_message_history.invoke([HumanMessage(content="Whats my name")], config=config)
response.content

'Your name is Madhu.'

In [22]:
#config = {"configurable": {"session_id": "dub1"}}

response = with_message_history.invoke([HumanMessage(content = "What is my name again?")], config=config,)
response.content

"I'm sorry, I do not have the ability to remember personal information about users."

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

In [27]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I assist you today?'

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [44]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [45]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [46]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [47]:
config = {"configurable": {"session_id": "abc11"}}

In [48]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'Hello Todd! How can I assist you today?'

In [49]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Your name is Todd. How can I assist you today, Todd?'

In [50]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm madhu. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Sure|,| Mad|hu|!| Here|'s| a| joke| for| you|:

|Why| don|'t| scientists| trust| atoms|?

|Because| they| make| up| everything|!| 😄||